In [1]:
import pickle

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objects as go

/tmp/ipykernel_5598/2137001591.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_5598/2137001591.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


#### Loading containers

In [5]:
with open('/mnt/c/Users/tahat/Downloads/PSA_boxpacking/Python_Repository/Data_for_study/containers_LaFabric.pickle', 'rb') as f:
    figures_prefaites = pickle.load(f)

In [6]:
def generate_interface_container(indice_container):
    """Generate interface of container correspond to indice
    
    keyword arguments:
    indice_container -- integer corresponding to the cardinal number of your container"""
    interface = html.Div(children = [
            html.Div(id = "div-value-diff-button{}".format(indice_container), children = 0, style = {"display":"none"}),
            dcc.Store(id = "value-number-box{}".format(indice_container)),

            html.Div(id = "div-backward{}".format(indice_container), style = {"display":"inline"}, children =\
                html.Button(id='submit-button-animation-backward{}'.format(indice_container), n_clicks = 0, children = 'Boite précédente ⬅️')),
            html.Div(id = "div-forward{}".format(indice_container), style = {"display":"inline"}, children = \
                html.Button(id='submit-button-animation-forward{}'.format(indice_container), n_clicks = 0, children='Boite suivante ➡️')),
            html.Div(id = "div-animation-button{}".format(indice_container), style = {"display":"inline"}, children = \
                html.Button(id='submit-button-animation-continue{}'.format(indice_container), n_clicks = 0, children='Animation 🎞️')),
            html.Div(id = "div-reset-button{}".format(indice_container), style = {"display":"inline"}, children = \
                html.Button(id='submit-button-reset{}'.format(indice_container), n_clicks = 0, children='Empty the container ❌')),
            html.Div(id = "div-fill-in-button{}".format(indice_container), style = {"display":"inline"}, children = \
                html.Button(id='submit-button-fill-in{}'.format(indice_container), n_clicks = 0, children='Fill in the container ✔️'))])
    return interface

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H4(children='Container Loading Problem 📦📦 - Container observation', style = {"textAlign":"center"}),
    
    ###Container 1 
    generate_interface_container(0),
    html.Div(id = "div-animation-interval0", style = {"display": "none", "visibility": "hidden"}, children =
            dcc.Interval(id = 'interval-component0', n_intervals = 0, max_intervals = 0)),
   
    dcc.Graph(id ='graph0', figure =  go.Figure(figures_prefaites[0])),
              
    ###Container 2 
    generate_interface_container(1),
    html.Div(id = "div-animation-interval1", style = {"display": "none", "visibility": "hidden"}, children =
            dcc.Interval(id = 'interval-component1', n_intervals = 0, max_intervals = 0)),
   
    dcc.Graph(id ='graph1', figure =  go.Figure(figures_prefaites[1])),

    ])

    
GLOBAL_VARS = {}
GLOBAL_VARS['ANIMATION_STATE_ENUM'] = {"STOP": "STOP", "ANIMATE": "ANIMATE"}

GLOBAL_VARS['NB_BOXES_N0'] = len(figures_prefaites[0].data) - 1
GLOBAL_VARS['ANIMATION_STATE_N0'] = GLOBAL_VARS['ANIMATION_STATE_ENUM']["STOP"]

GLOBAL_VARS['NB_BOXES_N1'] = len(figures_prefaites[1].data) - 1
GLOBAL_VARS['ANIMATION_STATE_N1'] = GLOBAL_VARS['ANIMATION_STATE_ENUM']["STOP"]


functions = []
for indice in range(2):

    @app.callback(Output('value-number-box{}'.format(indice), 'data'),
                  [Input('submit-button-animation-backward{}'.format(indice), 'n_clicks'),
                    Input('submit-button-animation-forward{}'.format(indice), 'n_clicks'), Input('submit-button-reset{}'.format(indice), 'n_clicks'),
                    Input('submit-button-fill-in{}'.format(indice), 'n_clicks')], [State("value-number-box{}".format(indice), 'data')])
    def calculate_diff(n_clicks_backward, n_clicks_forward, n_clicks_reset, n_clicks_fill_in, data_nb_boxes, indice=indice):
        """Make the button previous box, next box, empty container and fill in container work.
        change the animation_state to STOP of the corresponding figure if its called"""
        
#         print("Container {}".format(indice))
        global GLOBAL_VARS
        nb_boxes_dic = data_nb_boxes or {"boxes_display" : 0}

        if n_clicks_backward >= 1 or n_clicks_forward >= 1 or n_clicks_reset >= 1:
            GLOBAL_VARS['ANIMATION_STATE_N{}'.format(indice)] = GLOBAL_VARS['ANIMATION_STATE_ENUM']["STOP"]

            ## Remove a box
            if dash.callback_context.triggered[0]['prop_id'] == 'submit-button-animation-backward{}.n_clicks'.format(indice):
                ## if we are not at 0 we remove a box
                if GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] - 1 >= 0:
                    GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] -= 1

            ## add a box
            if dash.callback_context.triggered[0]['prop_id'] == 'submit-button-animation-forward{}.n_clicks'.format(indice):
                ## if we are not full we add a box
                if GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] + 1 <= len(figures_prefaites[indice].data) - 1:
                    GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] += 1

            ## Reset the container
            if dash.callback_context.triggered[0]['prop_id'] == 'submit-button-reset{}.n_clicks'.format(indice):
#                 if we are not full we add a box
                GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] = 0

            ## Fill in the container
            if dash.callback_context.triggered[0]['prop_id'] == 'submit-button-fill-in{}.n_clicks'.format(indice):
                ## if we are not full we add a box
                GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] = len(figures_prefaites[indice].data) - 1

        nb_boxes_dic["boxes_display"] = GLOBAL_VARS['NB_BOXES_N{}'.format(indice)]
        return nb_boxes_dic
    
    functions.append(calculate_diff)
    
     
    @app.callback([Output("graph{}".format(indice), 'figure'), Output('interval-component{}'.format(indice), 'disabled')],
                  [Input("value-number-box{}".format(indice), 'data'), Input('interval-component{}'.format(indice), 'n_intervals')])
    def animate_graphs(data, n_intervals,  indice=indice):
        """Get number of boxes that you want display and display it. Stop the animation if calculate_diff is called before"""

        global GLOBAL_VARS  
        is_interval_disabled = False

        ##To know if it's trigger by animation button or not
        if GLOBAL_VARS['ANIMATION_STATE_N{}'.format(indice)] == GLOBAL_VARS['ANIMATION_STATE_ENUM']["ANIMATE"]:
            if dash.callback_context.triggered[0]['prop_id'] == 'interval-component{}.n_intervals'.format(indice):
                if n_intervals > 0:
                    print(GLOBAL_VARS)
                    GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] = n_intervals
        else:
            is_interval_disabled = True
                            
        return go.Figure(figures_prefaites[indice].data[:GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] + 1]),is_interval_disabled

    functions.append(animate_graphs)

    

    @app.callback(Output("div-animation-interval{}".format(indice), "children"),
                  [Input("submit-button-animation-continue{}".format(indice), "n_clicks")])
    def make_animation(n_cliks,  indice=indice):
        """Make the animation run"""
        
        global GLOBAL_VARS

        if n_cliks >= 1:
            
            if GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] == len(figures_prefaites[indice].data) - 1:
                GLOBAL_VARS['NB_BOXES_N{}'.format(indice)] = 0
            
            GLOBAL_VARS['ANIMATION_STATE_N{}'.format(indice)] = GLOBAL_VARS['ANIMATION_STATE_ENUM']["ANIMATE"]
            return dcc.Interval(
                id = 'interval-component{}'.format(indice),
                interval = 1 * 900, # in milliseconds
                n_intervals = GLOBAL_VARS['NB_BOXES_N{}'.format(indice)],
                max_intervals = len(figures_prefaites[indice].data) - 1
            )
        else:
            return dcc.Interval(
                id = 'interval-component{}'.format(indice),
                interval = 1 * 900, # in milliseconds
                n_intervals = 0,
                max_intervals = 0
            )

    functions.append(make_animation)


app.run_server(host='127.0.0.1', port=8880, debug=True)